In [36]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
import skimage as sm
import skimage.morphology as morph
from skimage.io import imread, imsave
from skimage.util import img_as_float, random_noise
from skimage.transform import rotate, resize
from skimage.filters import gaussian as ski_gaussian
from skimage.feature import canny, corner_harris, corner_peaks
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d, fftconvolve
from scipy.signal import  gaussian as scipy_gaussian
from scipy.fft import fft2, fftshift, ifft2, ifftshift
from scipy.interpolate import griddata
from scipy.ndimage import map_coordinates, gaussian_filter

import numpy as np
import os
import timeit
import tensorflow as tf


os.chdir("../Mats")

In [44]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# 1

1.1

1.2

1.3

# 2

2.1

2.2

In [32]:
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, InputLayer
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
K.clear_session()
## Configure the network
# batch_size to train
batch_size = 20 * 256
# number of output classes
nb_classes = 135
# number of epochs to train
nb_epoch = 400

# number of convolutional filters to use
nb_filters = 20
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 5

model = Sequential([
    InputLayer(input_shape=(29, 29, 1)),
    Conv2D(filters=nb_filters, kernel_size=nb_conv, activation='relu'),
    MaxPool2D(pool_size=(nb_pool, nb_pool)),
    Dropout(0.5),
    Conv2D(filters=nb_filters, kernel_size=nb_conv, activation='relu'),
    MaxPool2D(pool_size=(nb_pool, nb_pool)),
    Dropout(0.25),
    Flatten(),
    Dense(units=4000, activation='relu'),
    Dense(units=nb_classes, activation='softmax'),
])
    
optimizer = Adam(lr=1e-4, epsilon=1e-08)

model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.load_weights('keras.h5') 

test = np.load(r'C:\Users\Jakob\Desktop\Uni\Kandidat\ISP\test.npz')
x_test= test['x_test']
y_test= test['y_test']
x_test = np.reshape(x_test, (x_test.shape[0], 29, 29, 1))


In [33]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', test_acc)

Test accuracy: 0.9576659202575684


2.3

In [40]:
def image_patches(image):
    padded_image = np.pad(image, ((14, 15), (14, 15)), 'constant')

    patches = sm.util.shape.view_as_windows(padded_image, (29, 29), step=1)

    return patches.reshape(-1, 29, 29, 1)

In [41]:
A = imread(r"C:\Users\Jakob\Desktop\Uni\Kandidat\ISP\SIP\Mats\test_images\image\1003_3_image.png")
testdata = image_patches(A)

In [52]:
patches = image_patches(A)
num_patches = patches.shape[0]

patches = patches / 255
class_probs = model.predict(patches)
seg_image = np.zeros((A.shape[0], A.shape[1]))
count = np.zeros((A.shape[0], A.shape[1]))

for i in range(num_patches):
    x = i % (A.shape[0]-28)
    y = i // (A.shape[0]-28)
    seg_patch = np.argmax(class_probs[i])
    seg_image[x:x+29, y:y+29] += seg_patch
    count[x:x+29, y:y+29] += 1

seg_image = seg_image/count
seg_image = resize(seg_image, A.shape, order=0, anti_aliasing=False)
seg_image = seg_image.astype(int)

2065/2065 [==============================] - 5s 3ms/step


In [53]:
plt.imshow(seg_image)

2.4

2.5